In [1]:
import cobra
import cplex
import numpy as np
import toolsNuevo
import time
import pickle
import math
import random

In [2]:
start=time.time()
numberInitialEFMs=500
path="./"
fileName1="EColiDecoupled.xml"
fileName2="EColi.xml"
model1=cobra.io.read_sbml_model(path+fileName1)
model2=cobra.io.read_sbml_model(path+fileName2)
S=cobra.util.array.create_stoichiometric_matrix(model1, array_type="dense", dtype=None)
m=len(S)
n=len(S[0])


'' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [3]:
A2=cobra.util.array.create_stoichiometric_matrix(model1, array_type="dense", dtype=None)
AA=[]
for i in range(len(A2)):
    indices=[]
    values=[]
    for j in range(len(A2[i])):
        if not A2[i][j]==0:
            indices.append(j)
            values.append(A2[i][j])
    AA.append([indices,values])

In [4]:
reduce=True
if reduce:
    A2=[AA[0],AA[1]]
    for i in range(2,len(AA)):
        A3=A2.copy()
        A3.append(AA[i])
        mm=len(A3)
        f=np.zeros(n)
        B=np.zeros(mm)
        B[mm-1]=1
        sol=toolsNuevo.resuelveDok(n,mm,f, A3, B)
        depende=True
        if len(sol)>1 or not sol[0]==3:
            depende=False
        if not depende:
            A2.append(AA[i])
    print(len(A2),np.linalg.matrix_rank(S))
    
    fileName="A2.txt"
    
    with open(fileName, 'wb') as filehandle:
        pickle.dump(A2, filehandle)
    


63 63


In [5]:
fileName=path+"reversibles.txt"
with open(fileName, 'rb') as filehandle:
    reversibles = pickle.load(filehandle)

fileName=path+"irreversibles.txt"
with open(fileName, 'rb') as filehandle:
    irreversibles = pickle.load(filehandle)
irreversibles=list(irreversibles)
    
fileName=path+"toDecoupled.txt"
with open(fileName, 'rb') as filehandle:
    toDecoupled = pickle.load(filehandle)

fileName=path+"toNonDecoupled.txt"
with open(fileName, 'rb') as filehandle:
    toNonDecoupled = pickle.load(filehandle)

fileName=path+"inverse.txt"
with open(fileName, 'rb') as filehandle:
    inverse = pickle.load(filehandle)

In [6]:
def create(k=1):
    global problem0, problem1, problem2, method
    
    B=np.zeros(len(A2))
    num_decision_var = n
    num_constraints = len(A2)
    problem0 = cplex.Cplex() 
    problem0.set_log_stream(None)
    method=problem0.parameters.lpmethod.values.primal
    problem0.parameters.lpmethod.set(method)
    problem0.set_error_stream(None)
    problem0.set_warning_stream(None)
    problem0.set_results_stream(None)
    problem0.variables.add(names= ["x"+str(i) for i in range(num_decision_var)])
    for i in range(num_decision_var):
        problem0.variables.set_lower_bounds(i, 0.0)

    for i in range(num_constraints):
        problem0.linear_constraints.add(
            lin_expr= [cplex.SparsePair(ind= A2[i][0], val= A2[i][1])],
            rhs= [B[i]],
            names = ["c"+str(i)],
            senses = ["E"]
        )

    problem1=cplex.Cplex(problem0)
    problem1.set_log_stream(None)
    problem1.parameters.lpmethod.set(method)
    problem1.parameters.advance.set(0)
    problem1.set_error_stream(None)
    problem1.set_warning_stream(None)
    problem1.set_results_stream(None)
    problem1.linear_constraints.add(
              rhs= [1],
              names = ["cc1"],
              senses = ["E"]
          )
    problem1.linear_constraints.set_linear_components([("cc1", cplex.SparsePair(ind= [28], val= [k]))])

    problem2=cplex.Cplex(problem0)
    problem2.set_log_stream(None)
    problem2.parameters.lpmethod.set(method)
    problem2.set_error_stream(None)
    problem2.set_warning_stream(None)
    problem2.set_results_stream(None)
    problem2.linear_constraints.add(
              rhs= [1],
              names = ["cc1"],
              senses = ["E"]
          )
    problem2.linear_constraints.add(
              rhs= [0],
              names = ["cc2"],
              senses = ["E"]
          )

In [7]:
def newSolution(n,m,f, list0, B):
    st=time.time()
    if len(list0)==1:
        myProblem=cplex.Cplex(problem1)
        myProblem.set_log_stream(None)
        myProblem.parameters.lpmethod.set(method)
        myProblem.set_error_stream(None)
        myProblem.set_warning_stream(None)
        myProblem.set_results_stream(None)
        mm=len(A2)+len(list0)
        myProblem.linear_constraints.set_linear_components([("cc1", cplex.SparsePair(ind= list0[0][0], val= list0[0][1]))])
    else:
        myProblem=cplex.Cplex(problem2)
        myProblem.set_log_stream(None)
        myProblem.parameters.lpmethod.set(method)
        myProblem.set_error_stream(None)
        myProblem.set_warning_stream(None)
        myProblem.set_results_stream(None)
        mm=len(A2)+len(list0)
        myProblem.linear_constraints.set_linear_components([("cc1", cplex.SparsePair(ind= list0[0][0], val= list0[0][1]))])
        myProblem.linear_constraints.set_linear_components([("cc2", cplex.SparsePair(ind= list0[1][0], val= list0[1][1]))])
  
    myProblem.objective.set_linear(f)
    myProblem.solve()  
    status=myProblem.solution.get_status()
    supports3=set()
    if status==1:       
        mode2=myProblem.solution.get_values()
        sop2=frozenset([j for j in range(n) if abs(mode2[j])>10**-10])
        sop3=frozenset([toNonDecoupled[j] for j in sop2])
        return sop3
    return sop2

In [8]:
def newSolution2(n,m,f, r, B,negatives):
    myProblem=cplex.Cplex(problem2)
    myProblem.set_log_stream(None)
    myProblem.parameters.lpmethod.set(method)
    myProblem.set_error_stream(None)
    myProblem.set_warning_stream(None)
    myProblem.set_results_stream(None)
    myProblem.linear_constraints.set_linear_components([("cc1", cplex.SparsePair(ind= [r], val= [1]))])
    myProblem.linear_constraints.set_linear_components([("cc2", cplex.SparsePair(ind= negatives, val= [1 for k in range(len(negatives))]))])
        
    myProblem.objective.set_linear(f)
    myProblem.solve()  
    status=myProblem.solution.get_status()
    supports3=set()
    if status==1:       
        mode2=myProblem.solution.get_values()
        sop2=frozenset([j for j in range(n) if abs(mode2[j])>10**-10])
        sop3=frozenset([toNonDecoupled[j] for j in sop2])
        return sop3
    else:
        return []
    

In [9]:
create()
supports=set()

for i in range(1,2):
    num=math.floor(3+3*i*n*0.01)
    
    for k in range(1000):
        r=random.choice(range(n))
        indices=[r]
        indices2=[]
        if r in inverse.keys():
            indices2=[inverse[r]]
        values=[]
        values2=[]
        for j in indices:
            values.append(1)
            values2.append(1)
        if len(indices2)==0:
            list0=[[indices,values]]
            B=[1]
        else:
            list0=[[indices,values],[indices2,values2]]
            B=[1,0]
        f=[]
        myIndices=random.sample(range(n),num)
        for j in myIndices:
            f.append([j,np.random.rand()]) 

        sop=newSolution(n,m,f, list0, B)
        supports.add(sop)
    print(i,num,len(supports))

1 6 460


In [10]:
functionLength=10
numberNegatives=2

tryouts=10000000

numberEFMs=10000

limit=500
counter=0

CS=dict()
for i in range(n):
    CS[i]=set()

objective=500
multiplier=1
times=dict()
for j in range(n):
    times[j]=0
for sop in supports:
    for j in sop:
        times[j]+=1
reactions=[j for j in range(n) if times[j]<0.9*len(supports) and times[j]>0.5*len(supports)]

supports=set()
st=time.time()
st0=time.time()


for j in range(tryouts):
    if len(supports)>objective*multiplier:
        times=dict()
        for j in range(n):
            times[j]=0
        for sop in supports:
            for j in sop:
                times[j]+=1
        reactions=[j for j in range(n) if times[j]<0.9*len(supports) and times[j]>0.5*len(supports)]
        multiplier+=1
        
    if j>0 and j%1000==0:
        print(j,len(supports),time.time()-st)
    if time.time()-st0>10*60 or len(supports)>numberEFMs:
        break
    
    r=random.choice(range(n))
    indices=[r]
    values=[1]
    B=[1,0]
    f=[]
    myIndices=random.sample(range(n),functionLength)
    for k in myIndices:
        f.append([k,np.random.rand()]) 
    indices2=[i for i in range(n) if not i==r]
    
    negatives=random.sample(reactions,numberNegatives)
    negatives2=[int(k) for k in negatives if not k==r and not k in CS[r]]
    if r in inverse.keys() and not inverse[r] in negatives2:
        negatives2.append(inverse[r])
    sop=newSolution2(n,m,f, r, B,list(set(negatives2)))
    if len(sop)>0:
        supports.add(sop)
    else:
        for k in negatives2:
            CS[r].add(k)

len(supports)

1000 527 1.81453275680542
2000 888 3.641371250152588
3000 1185 5.500187635421753
4000 1447 7.329241752624512
5000 1659 9.171928644180298
6000 1887 10.96805453300476
7000 2072 12.826683044433594
8000 2239 14.66804575920105
9000 2402 16.544166564941406
10000 2551 18.391321659088135
11000 2689 20.251057147979736
12000 2822 22.107985258102417
13000 2971 23.943139791488647
14000 3101 25.813488245010376
15000 3223 27.669910669326782
16000 3339 29.505793809890747
17000 3468 31.278421878814697
18000 3574 33.166953563690186
19000 3684 35.00135254859924
20000 3802 36.833974838256836
21000 3906 38.69510817527771
22000 3998 40.548526763916016
23000 4083 42.398754596710205
24000 4168 44.25362849235535
25000 4253 46.089324951171875
26000 4351 47.9286994934082
27000 4441 49.803712606430054
28000 4521 51.587769985198975
29000 4600 53.42470097541809
30000 4699 55.270946741104126
31000 4779 57.120707511901855
32000 4860 58.9622483253479
33000 4926 60.84179210662842
34000 4987 62.70351219177246
35000 506

10001

In [12]:
translate=True
if translate:
    translatedSupports=set()
    for sop in supports:
        sopT=frozenset([toNonDecoupled[j] for j in sop])
        translatedSupports.add(sopT)